In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


import tensorflow 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

from keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import csv
import numpy as np
from PIL import Image
from skimage import transform
from tensorflow.keras.models import load_model

2023-10-23 20:15:40.954760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 20:15:40.954854: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 20:15:40.955361: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 20:15:40.994800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:1

In [2]:
root_dir = 'archive/tw_food_101/tw_food_101'
train_dir = root_dir + '/train'
test_dir = root_dir + '/test'

In [16]:
#1. Cleaning and validation of images in the dataset
for subdir, _, files in os.walk(train_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        try:
            with Image.open(file_path) as img:
                if img.mode == 'P':  # Check for palette mode
                    img = img.convert('RGBA')
                    print('DONE!')
                img.verify()  # This will check for corrupt images
        except:
            print(f"Corrupt image {file_path} found and removed.")
            os.remove(file_path)

DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!


/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!
DONE!


**Setup Data Augmentation:**

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=40,
    zoom_range=0.2,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2  # reserve 20% of data for validation
)
 

**Setup Training and Validation Generators:**

In [4]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    color_mode = "rgb",
    batch_size = 8,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 42
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    color_mode = "rgb",
    batch_size=8,
    class_mode='categorical',
    subset='validation'
)


from sklearn.utils.class_weight import compute_sample_weight

classes = np.unique(train_generator.classes)
sample_weights = compute_sample_weight(class_weight='balanced', y=train_generator.classes)

class_weights = {}
for cls in classes:
    class_weights[cls] = sample_weights[train_generator.classes == cls].mean()

#print(class_weights)


Found 16323 images belonging to 101 classes.
Found 4049 images belonging to 101 classes.


In [5]:
#from tensorflow.keras.applications import EfficientNetB3
#base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
#base_model = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(101, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.000021)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


# Callbacks
#checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_weights_only=False, mode='max')
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, verbose=0, restore_best_weights=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=0.00001, mode='max')

2023-10-23 20:16:28.040495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:16:28.049162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:16:28.049207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:16:28.052298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:16:28.052510: I tensorflow/compile

In [ ]:
num_epochs = 50
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size 
STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size
history = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[early_stop, reduce_lr],
    #class_weight=class_weights
)

Epoch 1/50


2023-10-23 20:20:30.032167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-23 20:20:30.347821: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-23 20:20:30.807365: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-23 20:20:35.123078: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fefd81fd820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-23 20:20:35.123134: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2023-10-23 20:20:35.234700: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-23 20:20:35.759924: I ./tensorflow/compiler/jit/device_c

 268/2040 [==>...........................] - ETA: 6:19 - loss: 4.9912 - accuracy: 0.0271

/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


 658/2040 [========>.....................] - ETA: 4:38 - loss: 4.6335 - accuracy: 0.0492

/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1638/2040 [=======================>......] - ETA: 1:24 - loss: 3.9783 - accuracy: 0.1338

In [11]:
test_images = os.listdir(test_dir)

# Sort the test_images list
test_images = sorted(test_images, key=lambda x: int(os.path.splitext(x)[0]))

def load_img(filename, target_w=150, target_h=150):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (target_w, target_h, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

# Read images in order and make predictions
results = []
for filename in test_images:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, 150, 150)
    ret = model.predict(img)
    results.append(np.argmax(ret))
    
#test_dict = {}
#for root, dirs, files in os.walk(root_dir + '/test'): 
#    for filename in files:
#        test_id, file_ext = os.path.splitext(filename)
#        test_dict[test_id] = filename
    

#def load_img(filename, target_w=150, target_h=150):
#   np_image = Image.open(filename)
#   np_image = np.array(np_image).astype('float32')/255
 #  np_image = transform.resize(np_image, (target_w, target_h, 3))
 #  np_image = np.expand_dims(np_image, axis=0)
  # return np_image

# Read images in order and make predictions
#results = []
#for i in range(len(test_dict)):
#    img = load_img(root_dir + '/test/' + test_dict[str(i)], 150, 150)
 #   ret = model.predict(img)
 #   results.append(np.argmax(ret))
    

#test_datagen = ImageDataGenerator(rescale=1./255)

#test_generator = test_datagen.flow_from_directory(
  #  directory= test_dir,
  #  target_size=(128, 128),
   # batch_size=1,
   # class_mode=None,  # No labels
    #shuffle=False  # Keep data in same order as filenames for submission
#)

#predictions = model.predict(test_generator, steps=len(test_generator))
#predicted_classes = np.argmax(predictions, axis=1)


1/1 [==============================] - 0s 28ms/step


In [14]:
# Print results in CSV format and upload to Kaggle
with open('pred_results.csv', 'w') as f:
    f.write('Id,Category\n')
    for idx, prediction in enumerate(results):
        test_id, _ = os.path.splitext(test_images[idx])  # Extract file name without extension
        f.write(test_id + ',' + str(prediction) + '\n')

In [15]:
# Download your results!
from IPython.display import FileLink
FileLink('pred_results.csv')

/kaggle/working/pred_results.csv